# Capstone: Predicting IPL outcomes

# Notebook 1: Setting up the data

## Project summary

Sports betting is a billion-dollar industry. Cricket betting, albeit small compared to football or baseball, was still valued at USD10 billion annually with a further USD40 to USD50 billion in [illegal bets](http://www.espncricinfo.com/pakistan/content/story/535250.html) (often from India and Pakistan where betting is banned). Models that can consistently predict outcomes and beat the odds of bookmakers can therefore have great value.

This project aims to predict the outcome of Indian Premier League (IPL) matches. The IPL, a professional Twenty20 cricket league in India, is the richest tournament in cricket. It is contested annually in April and May. While the teams represent Indian cities (and in some cases a state), it attracts the best players from around the (cricket-playing) world.

The rules of cricket can seem quite involved, this [link](https://www.youtube.com/watch?v=AqtpNkMvj5Y) provides a 5 minute introduction 

The bulk of the data is available from a Kaggle [dataset](https://www.kaggle.com/manasgarg/ipl/data) which includes ball-by-ball information for each of the games between 2008 - 2017 (636 games in total) as well as a separate database with match level information. 

## Notebook 1: Summary
This notebook processes the data so that it can be inspected and visualized (Notebook 2) and modelled (Notebook 3). 
The original data consists of two datasets. The first – imported here as ‘IPL’ – contains ball-by-ball information of each game. For each ball it contains information on the batsman, the bowler, their respective team, and various measure based on the outcome of the (bowled) ball. The second dataset – imported as ‘matches’ contains game level information: the city, the venue, the teams and the outcome.
The structure (and the broad aims) of the notebook is as follows: <br>
1.	**Clean up the datasets:** This section ensure common spelling of team names, deals with some missing data, adapts some data to make it easier to use, and removes matches that were tied (or abandoned). The most important decision here is the removal of the entire 2009 season which was played in South Africa due to terrorist threats in India. While this makes an already small dataset even smaller, it is necessary as the conditions were not similar. <br>
2.	**Create a home team feature:** Home team advantage is a well-established part of sport. However, in the case of IPL it is somewhat complicated by the fluctuation of teams of years, the occasional sharing of home grounds, and some matches played in Dubai. This feature is ultimately more noise that hoped for, but is still likely to be valuable in the model.
3.	**Create intermediate features:** Create team IDs <br>
4.	**Summarize runs and wickets in a particular game:** This summarizes the ball by ball (IPL) dataset into the high level (matches) dataset for visualization in Notebook 2.<br>
5.	**Squad statistics:** The bulk of the processing of the data is to create team summary statistics based on individual information from the IPL set. Two new datasets (one for batting and one for bowling) are created containing the batting and bowling performance statistics of each player for a given season. These stats are then aggregated into team statistics which are then used as features. Note that this process is leaky: it includes information from games for the entire season. 



### Clean up the datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
% matplotlib inline
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [74]:
# Read in the two datasets
ipl = pd.read_csv('ipl_deliveries.csv')
matches = pd.read_csv('ipl_matches.csv')

In [75]:
ipl.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [76]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [77]:
# IPL DATAFRAME CLEANING

# Create wicket column
ipl['dismissed'] = ipl.player_dismissed.apply(lambda x: 1 if not pd.isnull(x) else 0)

# Create column if bowler gets the credit
def bowler_wicket(row):
    ''' Checks if there was a wicket, if yes, what type
    Returns 1 if bowler wicket, 0 otherwise'''
    # These wickets are not attributed to the bowler
    not_bowler = ['run out', 'retired hurt', 'obstructing the field']
    # The non-wicket deliveries are all NaN
    if row.dismissed==1:
        if row.dismissal_kind in not_bowler:
            return 0
        else:
            return 1
    else:
        return 0
                  
ipl['dismissed_bowler'] = ipl.apply(lambda row: bowler_wicket(row), axis=1) 

# Drop matches with no result
id_list = list(matches.loc[matches.winner.isnull(), 'id'])
ipl = ipl.loc[~(ipl.match_id.isin(id_list)), :]

# Drop tied games
tied_game = list(matches.loc[matches.result=='tie', 'id'])
ipl = ipl.loc[~(ipl.match_id.isin(tied_game)), :]

# Remove 2009 season played in South Africa (due to terrorist attacks in India)
sa_games = list(matches.loc[matches.season==2009, 'id'])
ipl = ipl.loc[~(ipl.match_id.isin(sa_games)), :]

# Remove games with DL method (shortened games due to rain)
dl_games = list(matches.loc[matches.dl_applied==1, 'id'])
ipl = ipl.loc[~(ipl.match_id.isin(dl_games)), :]

# Add season to IPL matches
ipl = ipl.merge(matches.loc[:, ['id', 'season']], left_on='match_id', right_on='id', how='left')

# MATCHES DATAFRAME CLEANING

# Remove no result matches (3)
matches.dropna(axis=0, subset = ['winner'], inplace=True)

# Remove all tied games
matches = matches.loc[matches['result']!='tie', :]

# Remove umpires (assume umpires don't cheat...)
matches.drop(['umpire1', 'umpire2', 'umpire3'], axis=1, inplace=True)

# Remove 2009 season played in South Africa (due to terrorist attacks in India)
matches = matches.loc[matches.season != 2009, :]

# Remove DL games (shortened due to rain)
matches = matches.loc[matches.dl_applied == 0, :]
matches.drop(['dl_applied'], axis=1, inplace=True)


print(len(ipl.match_id.unique()) == len(matches.id.unique()))

# Join Rising Pune Supergiants and Rising Pune Supergiant
ipl.loc[ipl.batting_team=='Rising Pune Supergiant', 'batting_team']= 'Rising Pune Supergiants'
ipl.loc[ipl.bowling_team=='Rising Pune Supergiant', 'bowling_team']= 'Rising Pune Supergiants'
matches.loc[matches.team1=='Rising Pune Supergiant', 'team1'] = 'Rising Pune Supergiants'
matches.loc[matches.team2=='Rising Pune Supergiant', 'team2'] = 'Rising Pune Supergiants'
matches.loc[matches.toss_winner=='Rising Pune Supergiant', 'toss_winner'] = 'Rising Pune Supergiants'
matches.loc[matches.winner=='Rising Pune Supergiant', 'winner'] = 'Rising Pune Supergiants'

# Nan cities are games played in Dubai
matches.loc[matches.city.isnull(), 'city'] = 'Dubai'

# Create winning columns to match team1 and team2
matches['team1_win'] = matches.apply(lambda x: 1 if x.winner==x.team1 else 0, axis=1)
matches['team2_win'] = matches.apply(lambda x: 1 if x.winner==x.team2 else 0, axis=1)

True


### Create a home team feature

In [78]:
# Home game variable is noisy. Impact may be lower than expected
# Multiple teams use sometimes share stadium
# Teams can have multiple home fields
# Could improve accuracy for Visakhapatnam, Cuttack, and Ranchi

home_city = {
    'Mumbai':'Mumbai Indians',
    'Bangalore':'Royal Challengers Bangalore',
    'Kolkata':'Kolkata Knight Riders',
    'Delhi':'Delhi Daredevils',
    'Hyderabad':['Sunrisers Hyderabad', 'Deccan Chargers'],
    'Chennai':'Chennai Super Kings',
    'Chandigarh': 'Kings XI Punjab',
    'Jaipur': 'Rajasthan Royals',
    'Pune': ['Rising Pune Supergiants', 'Pune Warriors'], #Multiple home teams
    'Ahmedabad': 'Rajasthan Royals',
    'Visakhapatnam':  ['Mumbai Indians','Rising Pune Supergiants'],  #Multiple home teams ,
    'Rajkot': 'Gujarat Lions',
    'Dharamsala': 'Kings XI Punjab',
    'Cuttack': ['Deccan Chargers', 'Kings XI Punjab', 'Kolkata Knight Riders'], #Multiple home teams 
    'Ranchi': ['Kolkata Knight Riders', 'Chennai Super Kings'], #Multiple home teams 
    'Abu Dhabi': 'Other', #No home team
    'Raipur': 'Delhi Daredevils',
    'Sharjah': 'Other', #No home team
    'Kochi': 'Kochi Tuskers Kerala',
    'Indore': 'Kings XI Punjab',
    'Kanpur': 'Gujarat Lions',
    'Nagpur': 'Deccan Chargers',
    'Dubai': 'Other' #No home team
}

# Use list to avoid index gaps in matches
team1list = list(matches.loc[:, 'team1'])
team2list = list(matches.loc[:, 'team2'])
city = list(matches.loc[:, 'city'])
home_team_list = []

# For each game
for i in range(len(team1list)):
    # Check if only one team listed as home ground
    if type(home_city[city[i]])==str:
        # Check if listed team actually played
        if home_city[city[i]] == team1list[i]:
            home_team_list.append(team1list[i])
        # Check if listed team actually played
        elif home_city[city[i]] == team2list[i]:
            home_team_list.append(team2list[i])
        else:
            home_team_list.append('No home team')
    # For cities with mulitple teams
    elif type(home_city[city[i]])==list:
        # Check if team is in list (if both teams use city as base then take first team)
        if team1list[i] in home_city[city[i]]:
            home_team_list.append(team1list[i])
        elif team2list[i] in home_city[city[i]]:
              home_team_list.append(team2list[i])
        else:
            home_team_list.append('No home team')
    else:
        home_team_list.append('No home team')
        
matches['home_team'] = home_team_list

### Some helpful (intermediate) features

In [79]:
# Create team IDs

# List of unique teams
teams = matches.team1.unique()
# Add 'no home team' to account for foreign venues 
teams = np.append(teams, np.array(['No home team']))

def teamID(teams):
    '''Gives every team in teams a unique value.
    Returns dict of all team IDs'''
    count = 0
    teamID = {}
    for team in teams:
        teamID[team] = count
        count += 1
    return teamID

# Generate team ID list
ID = teamID(teams)

# Apply to team1, team2 and toss_winner and winner, home-team in df
matches['team1_id'] = matches.team1.apply(lambda x: ID[x])
matches['team2_id'] = matches.team2.apply(lambda x: ID[x])
matches['toss_winner_id'] = matches.toss_winner.apply(lambda x: ID[x])
matches['winner_id'] = matches.winner.apply(lambda x: ID[x])
matches['home_team_id'] = matches.home_team.apply(lambda x: ID[x])

# Using abbreviations rather than (long!) full names

# matches.team1.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore',
#                        'Deccan Chargers','Chennai Super Kings','Rajasthan Royals','Delhi Daredevils',
#                        'Gujarat Lions','Kings XI Punjab', 'Sunrisers Hyderabad','Kochi Tuskers Kerala',
#                        'Pune Warriors','Rising Pune Supergiant']
#                 ,['MI','KKR','RCB','DC','CSK','RR','DD','GL','KXIP','SRH','KTK','PW','RPS'],inplace=True)
# matches.team2.replace

# delivery.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore','Deccan Chargers','Chennai Super Kings',
#                  'Rajasthan Royals','Delhi Daredevils','Gujarat Lions','Kings XI Punjab',
#                  'Sunrisers Hyderabad','Rising Pune Supergiants','Kochi Tuskers Kerala','Pune Warriors','Rising Pune Supergiant']
#                 ,['MI','KKR','RCB','DC','CSK','RR','DD','GL','KXIP','SRH','RPS','KTK','PW','RPS'],inplace=True)

In [80]:
ID

{'Chennai Super Kings': 8,
 'Deccan Chargers': 10,
 'Delhi Daredevils': 6,
 'Gujarat Lions': 2,
 'Kings XI Punjab': 7,
 'Kochi Tuskers Kerala': 11,
 'Kolkata Knight Riders': 5,
 'Mumbai Indians': 1,
 'No home team': 13,
 'Pune Warriors': 12,
 'Rajasthan Royals': 9,
 'Rising Pune Supergiants': 3,
 'Royal Challengers Bangalore': 4,
 'Sunrisers Hyderabad': 0}

### Runs and wickets in matches (only use for visualization)

In [97]:
# Helper functions to summarize runs and wickets from IPL df

def runs_in_1st_inn(ids, ipl):
    '''Returns total runs scored in 1st innings of game using ipl dataframe'''
    return ipl.loc[((ipl.match_id==ids) & (ipl.inning==1)), 'total_runs'].sum()

def bat_1st(ids, ipl):
    '''Returns team that batted first'''
    return ipl.loc[((ipl.match_id==ids) & (ipl.inning==1)), 'batting_team'].unique()[0]

def runs_in_2nd_inn(ids, ipl):
    '''Returns total runs scored in 1st innings of game using ipl dataframe'''
    return ipl.loc[((ipl.match_id==ids) & (ipl.inning==2)), 'total_runs'].sum()

def bat_2nd(ids, ipl):
    'Returns team that batted second'''
    return ipl.loc[((ipl.match_id==ids) & (ipl.inning==1)), 'bowling_team'].unique()[0]

def wickets_in_1st_inn(ids, ipl):
    '''Returns total wickets taken in 1st innings of game using ipl dataframe'''
    return ipl.loc[((ipl.match_id==ids) & (ipl.inning==1)), 'dismissed'].sum()

def wickets_in_2nd_inn(ids, ipl):
    '''Returns total wickets taken in 1st innings of game using ipl dataframe'''
    return ipl.loc[((ipl.match_id==ids) & (ipl.inning==2)), 'dismissed'].sum()



matches['runs_1st'] = matches.id.apply(lambda x: runs_in_1st_inn(x, ipl))
matches['runs_2nd'] = matches.id.apply(lambda x: runs_in_2nd_inn(x, ipl))
matches['wickets_1st'] = matches.id.apply(lambda x: wickets_in_1st_inn(x, ipl))
matches['wickets_2nd'] = matches.id.apply(lambda x: wickets_in_2nd_inn(x, ipl))
matches['bat_1st'] = matches.id.apply(lambda x: bat_1st(x, ipl))
matches['bat_2nd'] = matches.id.apply(lambda x: bat_2nd(x, ipl))

### Squad stats

In [99]:
# Can scrape squads, but easier to use ball-by-ball data from ipl df
# Use combine unique array of batsmen + unique array of bowlers as squad
# May miss players, e.g. Chennai2008 should be 26 players, here 19

# Teams in each year
teams08 = ipl.loc[ipl.season==2008, 'batting_team'].unique()
#No 2009 for now (played in SA)
#teams09 = ipl.loc[ipl.season==2009, 'batting_team'].unique()
teams10 = ipl.loc[ipl.season==2010, 'batting_team'].unique()
teams11 = ipl.loc[ipl.season==2011, 'batting_team'].unique()
teams12 = ipl.loc[ipl.season==2012, 'batting_team'].unique()
teams13 = ipl.loc[ipl.season==2013, 'batting_team'].unique()
teams14 = ipl.loc[ipl.season==2014, 'batting_team'].unique()
teams15 = ipl.loc[ipl.season==2015, 'batting_team'].unique()
teams16 = ipl.loc[ipl.season==2016, 'batting_team'].unique()
teams17 = ipl.loc[ipl.season==2017, 'batting_team'].unique()

def make_squads(ipl, teams, year):
    ''' Generate squads for each team in given year
    Returns dict of squads for each team for each year'''
    squad = {}
    for team in teams:
        squad[team] = np.unique(np.append(ipl.loc[(ipl.season==year) & (ipl.batting_team==team), 'batsman'].unique(),
                         ipl.loc[(ipl.season==year) & (ipl.bowling_team==team), 'bowler'].unique()))
    return squad

squad2008 = make_squads(ipl, teams08, 2008)
squad2010 = make_squads(ipl, teams10, 2010)
squad2011 = make_squads(ipl, teams11, 2011)
squad2012 = make_squads(ipl, teams12, 2012)
squad2013 = make_squads(ipl, teams13, 2013)
squad2014 = make_squads(ipl, teams14, 2014)
squad2015 = make_squads(ipl, teams15, 2015)
squad2016 = make_squads(ipl, teams16, 2016)
squad2017 = make_squads(ipl, teams17, 2017)

squadlist = [squad2008, squad2010, squad2011,
             squad2012, squad2013, squad2014,
             squad2015, squad2016, squad2017]

In [100]:
# Create a new dataframe with long form data, listing stats of each player in each season
# Note: Batting and bowling averages here ignore the impact of additional balls due to no-balls and wides
# This means batting and bowling averages will be slightly different to traditional measures

# Create df of batting stats of each player for a particular year

def make_batting_df(ipl, year):
    ''' Creates batting df of player and team by year
    Return df with player, team, and year'''
    
    #First need to create dataframe with each player as a row
    
    temp = ipl.loc[ipl.season == year, ['batsman', 'batting_team', 'batsman_runs']]
    player_df = pd.pivot_table(temp,
                               index=['batsman', 'batting_team'],
                               values='batsman_runs', aggfunc=(sum, 'count'))
    # Reset index from pivot_table
    player_df.reset_index(inplace=True)
    # Rename columns
    player_df.columns = ['batsman', 'batting_team', 'balls_faced', 'runs_scored']
    # Either player can be dismissed, so need to build another df for dismissals
    player_out = ipl.loc[ipl.season==year, 'player_dismissed'].value_counts().to_frame()
    player_out.reset_index(inplace=True)
    # Merge dfs and drop right_merge column (duplicate)
    players = player_df.merge(player_out, left_on='batsman',
                              right_on='index', how='left').drop('index', axis=1)
    # Add year back in (useful when all these dfs are joined)
    players['season']=year
    
    #Now that we have all the players, compute some stats
    
    # Need to account for player_dismissed = 0
    players['avgs'] = players.apply(
        lambda x: x.runs_scored/x.player_dismissed if x.player_dismissed>0 else x.runs_scored, axis=1)
    # Strike rate given per hunderd balls
    players['strike_rate'] = players.apply(
        lambda x: (x.runs_scored/x.balls_faced)*100 if x.balls_faced>0 else 0, axis=1)
    return players

# Create df of bowling stats of each player for a particular year

def make_bowling_df(ipl, year):
    ''' Creates bowling df of player and team by year
    Return df with player, team, and year'''
    
    # Change ipl to show each player as a row
    
    temp = ipl.loc[ipl.season == year, ['bowler', 'total_runs', 'dismissed_bowler', 'bowling_team']]
    # Pivot on players
    player_df = pd.pivot_table(temp,
                               index=['bowler', 'bowling_team'],
                               values=['total_runs', 'dismissed_bowler'],
                               aggfunc=(sum, 'count'))
    # Rename columns
    player_df.columns = ['balls_bowled', 'wickets', 'balls_bowled_2', 'runs']
    # Reset index from pivot table
    player_df.reset_index(inplace=True)
    # Add in year (useful when all dfs are combined)
    player_df['season'] = year
    
    #Now for some stats
    
    # That pesky 0 makes this long
    player_df['bowling_avg'] = player_df.apply(
        lambda x: x.runs/x.wickets if x.wickets>0 else 0, axis=1)
    # Economy rate is measured by over (hence *6) 
    player_df['economy'] = player_df.apply(
        lambda x: (x.runs/x.balls_bowled)*6 if x.balls_bowled>0 else 0, axis=1)
    player_df['strike_rate'] = player_df.apply(
        lambda x: x.balls_bowled/x.wickets if x.wickets>0 else 0, axis=1)
    return player_df

In [101]:
# Create dfs for batting and bowling for each year
year_list = [2008, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]


# Bowling
frames = []
for year in year_list:
    temp = make_bowling_df(ipl, year)
    frames.append(temp)
    
bowling_stats  = pd.concat(frames)

# Batting
frames = []
for year in ipl.season.unique():
    temp = make_batting_df(ipl, year)
    frames.append(temp)
    
batting_stats  = pd.concat(frames)

In [102]:
bowling_stats.to_csv(r'C:\Users\User\Documents\Python_scripts\Thinkful\IPL_bowling.csv', index=False)
batting_stats.to_csv(r'C:\Users\User\Documents\Python_scripts\Thinkful\IPL_batting.csv', index=False)

In [103]:
# Now functions to create seperate stats for each team for each year
# These functions will be used to update the matches df

year_list = [2008, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
# Teams per season calculated above, now combine as list
team_list = [teams08, teams10, teams11,
             teams12, teams13, teams14, teams15, teams16, teams17]

def team_bowling_stats(bowling_stats, year_list, team_list):
    ''' Takes a df of bowling stats (all players, all teams, all years)
    Takes a list of years to iterate over
    Calculates team stats based on best players in squad
    Returns dataframe of team bowling stats for every year'''
    
    # Create the df 
    team_bowling_stats = pd.DataFrame(columns=['team','season','avg','strike','economy'])
    print(team_bowling_stats.head())
    # Number of balls bowled to be considered mainstream bowler (roughly 10%) 
    cut_off = 72
    
    # 2008 and 2010 use stats from **that** year (treated as initial years)
    for i, year in enumerate(year_list):
        # List of teams that played in a particular year
        print("Calculating for: ", year)
        teams = team_list[i]
        #if (year==2008) | (year==2010):
        # Nested for loop. Let's pretend you didn't see this
        for team in teams:
            print("Calculating for team: ", team)
            current_team = bowling_stats.loc[(bowling_stats.bowling_team==team) &
                                             (bowling_stats.season==year),
                                             ['bowler', 'balls_bowled', 'bowling_avg',
                                              'economy', 'strike_rate']]
            current_team['topbowlers'] = current_team.balls_bowled > 72
            avg = current_team.loc[current_team.topbowlers==True, 'bowling_avg'].mean()
            strike = current_team.loc[current_team.topbowlers==True, 'strike_rate'].mean()
            economy = current_team.loc[current_team.topbowlers==True, 'economy'].mean()
            final = pd.Series({'team': team, 'season': year,
                               'avg': avg, 'strike': strike,
                               'economy': economy})
            print("final scores of for {} in {}".format(team, year))
            print(final)
            team_bowling_stats = team_bowling_stats.append(final, ignore_index=True)
    return team_bowling_stats
    
team_bowling = team_bowling_stats(bowling_stats, year_list, team_list)

Empty DataFrame
Columns: [team, season, avg, strike, economy]
Index: []
Calculating for:  2008
Calculating for team:  Kolkata Knight Riders
final scores of for Kolkata Knight Riders in 2008
avg                      33.8591
economy                  8.13756
season                      2008
strike                   24.5791
team       Kolkata Knight Riders
dtype: object
Calculating for team:  Royal Challengers Bangalore
final scores of for Royal Challengers Bangalore in 2008
avg                            45.7177
economy                        8.00495
season                            2008
strike                         33.8158
team       Royal Challengers Bangalore
dtype: object
Calculating for team:  Chennai Super Kings
final scores of for Chennai Super Kings in 2008
avg                    37.4867
economy                8.40939
season                    2008
strike                 27.6309
team       Chennai Super Kings
dtype: object
Calculating for team:  Kings XI Punjab
final scores of 

Calculating for:  2013
Calculating for team:  Delhi Daredevils
final scores of for Delhi Daredevils in 2013
avg                 35.1731
economy             7.64262
season                 2013
strike              27.7056
team       Delhi Daredevils
dtype: object
Calculating for team:  Kolkata Knight Riders
final scores of for Kolkata Knight Riders in 2013
avg                      24.4395
economy                  7.03987
season                      2013
strike                   20.6873
team       Kolkata Knight Riders
dtype: object
Calculating for team:  Royal Challengers Bangalore
final scores of for Royal Challengers Bangalore in 2013
avg                            35.0661
economy                        7.99236
season                            2013
strike                         25.8452
team       Royal Challengers Bangalore
dtype: object
Calculating for team:  Mumbai Indians
final scores of for Mumbai Indians in 2013
avg                29.938
economy           7.63571
season         

In [104]:
def team_batting_stats(batting_stats, year_list, team_list):
    ''' Takes a df of bowling stats (all players, all teams, all years)
    Takes a list of years to iterate over
    Calculates team stats based on best players in squad
    Returns dataframe of team bowling stats for every year'''
    
    # Create the df 
    team_batting_stats = pd.DataFrame(columns=['team','season','runs_scored','avg','strike_rate'])
    # Take only the top 7 batsmen 
    cut_off = 7
    # 2008 and 2010 use stats from **that** year (treated as initial years)
    for i, year in enumerate(year_list):
        # List of teams that played in a particular year
        print("Calculating for: ", year)
        teams = team_list[i]
        #if (year==2008) | (year==2010):
        # Nested for loop. Let's pretend you didn't see this
        for team in teams:
            print("Calculating for team: ", team)
            current_team = batting_stats.loc[(batting_stats.batting_team==team) &
                                             (batting_stats.season==year),
                                             ['batsman', 'runs_scored', 'avgs',
                                              'strike_rate']]
            current_team = current_team.sort_values('runs_scored', ascending=False)
            totalruns_avg = current_team.iloc[0:cut_off, 1].mean() 
            avg = current_team.iloc[0:cut_off, 2].mean()
            strike = current_team.iloc[0:cut_off, 3].mean()
            final = pd.Series({'team': team, 'season': year,
                               'runs_scored': totalruns_avg,
                               'avg': avg, 'strike_rate': strike,
                               })
            print("final scores of for {} in {}".format(team, year))
            print(final)
            team_batting_stats = team_batting_stats.append(final, ignore_index=True)
    return team_batting_stats
    
team_batting = team_batting_stats(batting_stats, year_list, team_list)

Calculating for:  2008
Calculating for team:  Kolkata Knight Riders
final scores of for Kolkata Knight Riders in 2008
avg                          29.8374
runs_scored                      192
season                          2008
strike_rate                  130.142
team           Kolkata Knight Riders
dtype: object
Calculating for team:  Royal Challengers Bangalore
final scores of for Royal Challengers Bangalore in 2008
avg                                23.8433
runs_scored                        191.571
season                                2008
strike_rate                        124.997
team           Royal Challengers Bangalore
dtype: object
Calculating for team:  Chennai Super Kings
final scores of for Chennai Super Kings in 2008
avg                        45.5469
runs_scored                272.571
season                        2008
strike_rate                136.878
team           Chennai Super Kings
dtype: object
Calculating for team:  Kings XI Punjab
final scores of for Kings XI

Calculating for team:  Royal Challengers Bangalore
final scores of for Royal Challengers Bangalore in 2012
avg                                31.7876
runs_scored                        312.143
season                                2012
strike_rate                        125.426
team           Royal Challengers Bangalore
dtype: object
Calculating for team:  Deccan Chargers
final scores of for Deccan Chargers in 2012
avg                      36.87
runs_scored            277.429
season                    2012
strike_rate            121.053
team           Deccan Chargers
dtype: object
Calculating for:  2013
Calculating for team:  Delhi Daredevils
final scores of for Delhi Daredevils in 2013
avg                     23.6296
runs_scored             221.571
season                     2013
strike_rate             111.224
team           Delhi Daredevils
dtype: object
Calculating for team:  Kolkata Knight Riders
final scores of for Kolkata Knight Riders in 2013
avg                          22.652

In [105]:
# Some more helper functions
# This time taking data from the team_batting or team_bolwing df to the matches df

# Batting
def team_batting_avg(team_batting, team_bat, year):
    return float(team_batting.loc[(team_batting.team==team_bat) & (team_batting.season==year),
                           'avg'])

def team_batting_runs(team_batting, team_bat, year):
    return float(team_batting.loc[(team_batting.team==team_bat) & (team_batting.season==year),
                           'runs_scored'])

def team_batting_strike(team_batting, team_bat, year):
        return float(team_batting.loc[(team_batting.team==team_bat) & (team_batting.season==year),
                           'strike_rate'])

# Bowling 
def team_bowling_avg(team_bowling, team_bowl, year):
     return float(team_bowling.loc[(team_bowling.team==team_bowl) & (team_bowling.season==year),
                           'avg'])

def team_bowling_strike(team_bowling, team_bowl, year):
     return float(team_bowling.loc[(team_bowling.team==team_bowl) & (team_bowling.season==year),
                           'strike'])

def team_bowling_economy(team_bowling, team_bowl, year):
     return float(team_bowling.loc[(team_bowling.team==team_bowl) & (team_bowling.season==year),
                           'economy'])
    

In [106]:
# Finally! Adding it all back into matches

# New batting features
matches['team1_bat_avg'] = matches.apply(lambda x: team_batting_avg(team_batting, x.team1, x.season), axis=1)
matches['team2_bat_avg'] = matches.apply(lambda x: team_batting_avg(team_batting, x.team2, x.season), axis=1)
matches['team1_total_runs'] = matches.apply(lambda x: team_batting_runs(team_batting, x.team1, x.season), axis=1)
matches['team2_total_runs'] = matches.apply(lambda x: team_batting_runs(team_batting, x.team2, x.season), axis=1)
matches['team1_bat_strike'] = matches.apply(lambda x: team_batting_strike(team_batting, x.team1, x.season), axis=1)
matches['team2_bat_strike'] = matches.apply(lambda x: team_batting_strike(team_batting, x.team2, x.season), axis=1)

# New bowling features
matches['team1_bowl_avg'] = matches.apply(lambda x: team_bowling_avg(team_bowling, x.team1, x.season), axis=1)
matches['team2_bowl_avg'] = matches.apply(lambda x: team_bowling_avg(team_bowling, x.team2, x.season), axis=1)
matches['team1_bowl_strike'] = matches.apply(lambda x: team_bowling_strike(team_bowling, x.team1, x.season), axis=1)
matches['team2_bowl_strike'] = matches.apply(lambda x: team_bowling_strike(team_bowling, x.team2, x.season), axis=1)
matches['team1_bowl_econ'] = matches.apply(lambda x: team_bowling_economy(team_bowling, x.team1, x.season), axis=1)
matches['team2_bowl_econ'] = matches.apply(lambda x: team_bowling_economy(team_bowling, x.team2, x.season), axis=1)


In [107]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,winner,...,team1_total_runs,team2_total_runs,team1_bat_strike,team2_bat_strike,team1_bowl_avg,team2_bowl_avg,team1_bowl_strike,team2_bowl_strike,team1_bowl_econ,team2_bowl_econ
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,Sunrisers Hyderabad,...,283.000000,210.571429,140.141978,124.361994,64.266562,33.864831,41.861844,24.179365,8.415585,8.128939
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiants,Rising Pune Supergiants,field,normal,Rising Pune Supergiants,...,294.285714,322.000000,132.086532,130.395824,29.264901,24.644323,23.391817,20.486179,7.520773,7.280775
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,Kolkata Knight Riders,...,287.142857,290.142857,143.623532,142.599870,56.936111,30.707516,36.400000,22.357734,9.224123,8.291170
3,4,2017,Indore,2017-04-08,Rising Pune Supergiants,Kings XI Punjab,Kings XI Punjab,field,normal,Kings XI Punjab,...,322.000000,259.428571,130.395824,139.994981,24.644323,26.471885,20.486179,19.738987,7.280775,8.188582
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,Royal Challengers Bangalore,...,210.571429,257.857143,124.361994,136.818079,33.864831,34.731250,24.179365,24.727083,8.128939,8.234443


## Save for next notebook

In [110]:
matches.to_csv(r'C:\Users\User\Documents\Python_scripts\Thinkful\matches_updated.csv', index=False)